In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
import gensim.downloader as api

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df = pd.read_csv("/content/spam.csv", encoding='latin-1')[['v1', 'v2']]
df.columns = ['Label', 'Message']
df['Label'] = df['Label'].map({'spam': 1, 'ham': 0})

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    import nltk
    nltk.download('punkt', quiet=True)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

df['Tokens'] = df['Message'].apply(preprocess)

In [ ]:
w2v_model = api.load('word2vec-google-news-300')

In [ ]:
def vectorize(tokens, model, dim=300):
    vectors = [model[word] for word in tokens if word in model]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

df['Vector'] = df['Tokens'].apply(lambda x: vectorize(x, w2v_model))
X = np.vstack(df['Vector'].values)
y = df['Label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9443946188340807


In [ ]:
def predict_message_class(model, w2v_model, message):
    tokens = preprocess(message)
    vec = vectorize(tokens, w2v_model).reshape(1, -1)
    pred = model.predict(vec)[0]
    return "spam" if pred == 1 else "ham"

In [ ]:
print(predict_message_class(clf, w2v_model, "Free entry in 2 a weekly competition to win tickets!"))
print(predict_message_class(clf, w2v_model, "Hey, I will call you later"))

spam
ham
